In [1]:
import geopandas as gpd

In [2]:
file_path = r'data\WalkabilityIndex\Natl_WI.gdb'

# Assuming gdf is already defined
gdf = gpd.read_file(file_path)

# Inspect the dataframe
gdf.head(n=1)

,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,CBSA_Name,...,D3B,D4A,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area,geometry
0,481130078254,481130078254,48,113,007825,4,206,"Dallas-Fort Worth, TX-OK",19100,"Dallas-Fort Worth-Arlington, TX",...,115.981747,362.1,6.0,14.0,15.0,17.0,14.0,3110.36082,297836.08309,"MULTIPOLYGON (((-68983.316 1091325.734, -68981..."


In [3]:
# Count the GEOID20 per CBSA_Name. Convert to dataframe 'blocks
blocks = gdf.groupby('CBSA_Name')['GEOID20'].count().reset_index()

# Rename the columns
blocks.columns = ['CBSA_Name', 'Count of Blocks']

# Sort the values on 'Count of Blocks
blocks = blocks.sort_values(by='Count of Blocks', ascending=False)
blocks.head()

,CBSA_Name,Count of Blocks
604,"New York-Newark-Jersey City, NY-NJ-PA",14376
498,"Los Angeles-Long Beach-Anaheim, CA",8248
162,"Chicago-Naperville-Elgin, IL-IN-WI",6590
664,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",4306
211,"Dallas-Fort Worth-Arlington, TX",4128


In [9]:
# Interesting. the NatWalkIndex is on a scale of 1 to 20, not 0 to 100 like the WalkScore.
gdf['NatWalkInd'].describe()

count    220739.000000
mean          9.541667
std           4.373924
min           1.000000
25%           5.833333
50%           9.166667
75%          13.166667
max          20.000000
Name: NatWalkInd, dtype: float64

In [7]:
# Show me the NatWalkIndex for CBSA_Name 'New York-Newark-Jersey City, NY-NJ-PA'
nyc_walk = gdf[gdf['CBSA_Name'] == 'New York-Newark-Jersey City, NY-NJ-PA'][['NatWalkInd']].reset_index(drop=True)
nyc_walk.describe()

,NatWalkInd
count,14376.000000
mean,12.582974
std,3.634793
min,1.000000
25%,10.333333
50%,13.333333
75%,15.333333
max,20.000000
